# Overview

This Colab helps validate a training set for the NN classifier soultion.

# Step 0: Start Colab

Connect the Colab to hosted Python3 runtime (check top-right corner) and then install required dependencies.

In [2]:
!pip install pillow
!pip install matplotlib
!pip install numpy
!pip install opencv-python
!pip install tqdm
!pip install mediapipe==0.8.10

import torch
import torch.optim as optim
import torch.nn as nn
import random
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 275 kB/s 


# Codebase

## Pose embedding

In [4]:
class FullBodyPoseEmbedder(object):
  """Converts 3D pose landmarks into 3D embedding."""

  def __init__(self, torso_size_multiplier=2.5):
    # Multiplier to apply to the torso to get minimal body size.
    self._torso_size_multiplier = torso_size_multiplier

    # Names of the landmarks as they appear in the prediction.
    self._landmark_names = [
        'nose',
        'left_eye_inner', 'left_eye', 'left_eye_outer',
        'right_eye_inner', 'right_eye', 'right_eye_outer',
        'left_ear', 'right_ear',
        'mouth_left', 'mouth_right',
        'left_shoulder', 'right_shoulder',
        'left_elbow', 'right_elbow',
        'left_wrist', 'right_wrist',
        'left_pinky', 'right_pinky',
        'left_index', 'right_index',
        'left_thumb', 'right_thumb',
        'left_hip', 'right_hip',
        'left_knee', 'right_knee',
        'left_ankle', 'right_ankle',
        'left_heel', 'right_heel',
        'left_foot_index', 'right_foot_index',
    ]

  def __call__(self, landmarks):
    """Normalizes pose landmarks and converts to embedding
    
    Args:
      landmarks - NumPy array with 3D landmarks of shape (N, 3).

    Result:
      Numpy array with pose embedding of shape (M, 3) where `M` is the number of
      pairwise distances defined in `_get_pose_distance_embedding`.
    """
    assert landmarks.shape[0] == len(self._landmark_names), 'Unexpected number of landmarks: {}'.format(landmarks.shape[0])

    # Get pose landmarks.
    landmarks = np.copy(landmarks)

    # Normalize landmarks.
    landmarks = self._normalize_pose_landmarks(landmarks)

    # Get embedding.
    embedding = self._get_pose_distance_embedding(landmarks)

    return embedding

  def _normalize_pose_landmarks(self, landmarks):
    """Normalizes landmarks translation and scale."""
    landmarks = np.copy(landmarks)

    # Normalize translation.
    pose_center = self._get_pose_center(landmarks)
    landmarks -= pose_center

    # Normalize scale.
    pose_size = self._get_pose_size(landmarks, self._torso_size_multiplier)
    landmarks /= pose_size
    # Multiplication by 100 is not required, but makes it easier to debug.
    landmarks *= 100

    return landmarks

  def _get_pose_center(self, landmarks):
    """Calculates pose center as point between hips."""
    left_hip = landmarks[self._landmark_names.index('left_hip')]
    right_hip = landmarks[self._landmark_names.index('right_hip')]
    center = (left_hip + right_hip) * 0.5
    return center

  def _get_pose_size(self, landmarks, torso_size_multiplier):
    """Calculates pose size.
    
    It is the maximum of two values:
      * Torso size multiplied by `torso_size_multiplier`
      * Maximum distance from pose center to any pose landmark
    """
    # This approach uses only 2D landmarks to compute pose size.
    landmarks = landmarks[:, :2]

    # Hips center.
    left_hip = landmarks[self._landmark_names.index('left_hip')]
    right_hip = landmarks[self._landmark_names.index('right_hip')]
    hips = (left_hip + right_hip) * 0.5

    # Shoulders center.
    left_shoulder = landmarks[self._landmark_names.index('left_shoulder')]
    right_shoulder = landmarks[self._landmark_names.index('right_shoulder')]
    shoulders = (left_shoulder + right_shoulder) * 0.5

    # Torso size as the minimum body size.
    torso_size = np.linalg.norm(shoulders - hips)

    # Max dist to pose center.
    pose_center = self._get_pose_center(landmarks)
    max_dist = np.max(np.linalg.norm(landmarks - pose_center, axis=1))

    return max(torso_size * torso_size_multiplier, max_dist)

  def _get_pose_distance_embedding(self, landmarks):
    """Converts pose landmarks into 3D embedding.

    We use several pairwise 3D distances to form pose embedding. All distances
    include X and Y components with sign. We differnt types of pairs to cover
    different pose classes. Feel free to remove some or add new.
    
    Args:
      landmarks - NumPy array with 3D landmarks of shape (N, 3).

    Result:
      Numpy array with pose embedding of shape (M, 3) where `M` is the number of
      pairwise distances.
    """
    embedding = np.array([
        # One joint.

        self._get_distance(
            self._get_average_by_names(landmarks, 'left_hip', 'right_hip'),
            self._get_average_by_names(landmarks, 'left_shoulder', 'right_shoulder')),

        self._get_distance_by_names(landmarks, 'left_shoulder', 'left_elbow'),
        self._get_distance_by_names(landmarks, 'right_shoulder', 'right_elbow'),

        self._get_distance_by_names(landmarks, 'left_elbow', 'left_wrist'),
        self._get_distance_by_names(landmarks, 'right_elbow', 'right_wrist'),

        self._get_distance_by_names(landmarks, 'left_hip', 'left_knee'),
        self._get_distance_by_names(landmarks, 'right_hip', 'right_knee'),

        self._get_distance_by_names(landmarks, 'left_knee', 'left_ankle'),
        self._get_distance_by_names(landmarks, 'right_knee', 'right_ankle'),

        # Two joints.

        self._get_distance_by_names(landmarks, 'left_shoulder', 'left_wrist'),
        self._get_distance_by_names(landmarks, 'right_shoulder', 'right_wrist'),

        self._get_distance_by_names(landmarks, 'left_hip', 'left_ankle'),
        self._get_distance_by_names(landmarks, 'right_hip', 'right_ankle'),

        # Three joints.

        self._get_distance_by_names(landmarks, 'left_hip', 'left_wrist'),
        self._get_distance_by_names(landmarks, 'right_hip', 'right_wrist'),

        self._get_distance_by_names(landmarks, 'left_elbow', 'left_knee'),
        self._get_distance_by_names(landmarks, 'right_elbow', 'right_knee'),

        self._get_distance_by_names(landmarks, 'left_shoulder', 'left_ankle'),
        self._get_distance_by_names(landmarks, 'right_shoulder', 'right_ankle'),

        # Cross body.

        self._get_distance_by_names(landmarks, 'left_elbow', 'right_elbow'),
        self._get_distance_by_names(landmarks, 'left_knee', 'right_knee'),

        self._get_distance_by_names(landmarks, 'left_wrist', 'right_wrist'),
        self._get_distance_by_names(landmarks, 'left_ankle', 'right_ankle'),

        # Body bent direction.

        self._get_distance(
            self._get_average_by_names(landmarks, 'left_wrist', 'left_ankle'),
            landmarks[self._landmark_names.index('left_hip')]),
        self._get_distance(
            self._get_average_by_names(landmarks, 'right_wrist', 'right_ankle'),
            landmarks[self._landmark_names.index('right_hip')]),
    ])

    # embedding = np.array([
    #     # One side.
    #     self._get_angle_by_names(landmarks, 'left_wrist', 'left_elbow', 'left_shoulder'),
    #     self._get_angle_by_names(landmarks, 'right_wrist', 'right_elbow', 'right_shoulder'),
    #     self._get_angle_by_names(landmarks, 'left_shoulder', 'left_hip', 'left_knee'),
    #     self._get_angle_by_names(landmarks, 'right_shoulder', 'right_hip', 'right_knee'),
    #     self._get_angle_by_names(landmarks, 'left_hip', 'left_knee', 'left_ankle'),
    #     self._get_angle_by_names(landmarks, 'right_hip', 'right_knee', 'right_ankle'),
    #     self._get_angle_by_names(landmarks, 'left_elbow', 'left_shoulder', 'left_hip'),
    #     self._get_angle_by_names(landmarks, 'right_elbow', 'right_shoulder', 'right_hip'),
        
    #     # Cross body.
    #     self._get_angle_by_names(landmarks, 'left_shoulder', 'right_shoulder', 'right_hip'),
    #     self._get_angle_by_names(landmarks, 'right_shoulder', 'left_shoulder', 'left_hip'),
    #     self._get_angle_by_names(landmarks, 'left_shoulder', 'left_hip', 'right_hip'),
    #     self._get_angle_by_names(landmarks, 'right_shoulder', 'right_hip', 'left_hip'),
    #     self._get_angle_by_names(landmarks, 'left_hip', 'right_hip', 'right_knee'),
    #     self._get_angle_by_names(landmarks, 'right_hip', 'left_hip', 'left_knee'),
    # ])

    return embedding

  def _get_average_by_names(self, landmarks, name_from, name_to):
    lmk_from = landmarks[self._landmark_names.index(name_from)]
    lmk_to = landmarks[self._landmark_names.index(name_to)]
    return (lmk_from + lmk_to) * 0.5

  def _get_distance_by_names(self, landmarks, name_from, name_to):
    lmk_from = landmarks[self._landmark_names.index(name_from)]
    lmk_to = landmarks[self._landmark_names.index(name_to)]
    return self._get_distance(lmk_from, lmk_to)
  
  def _get_distance(self, lmk_from, lmk_to):
    return lmk_to - lmk_from
  
  def _get_angle_by_names(self, landmarks, name_from, name_center, name_to):
    lmk_from = landmarks[self._landmark_names.index(name_from)]
    lmk_center = landmarks[self._landmark_names.index(name_center)]
    lmk_to = landmarks[self._landmark_names.index(name_to)]
    return self._get_angle(lmk_from, lmk_center, lmk_to)

  def _get_angle(self, lmk_from, lmk_center, lmk_to):
    vec_from = (lmk_from - lmk_center) * (1., 1., 0.2)
    vec_to = (lmk_to - lmk_center) * (1., 1., 0.2)
    
    return np.dot(vec_from, vec_to) / (np.linalg.norm(vec_from) * np.linalg.norm(vec_to))

## Pose classification

In [5]:
class PoseSample(object):

  def __init__(self, name, landmarks, class_name, embedding):
    self.name = name
    self.landmarks = landmarks
    self.class_name = class_name
    self.embedding = embedding

In [21]:
import torch.nn as nn
class Net_3(nn.Module):
  def __init__(self,input,H0,H1,output):
    super(Net_3,self).__init__()
    self.linear1=nn.Linear(input,H0)
    self.linear2=nn.Linear(H0,H1)
    self.linear3=nn.Linear(H1,output)
  
  def forward(self,x):
    x=torch.sigmoid(self.linear1(x))  
    x=torch.sigmoid(self.linear2(x))  
    x=torch.sigmoid(self.linear3(x))
    return x

import torch.nn as nn
class Net_2(nn.Module):
  def __init__(self,input,H,output):
    super(Net_2,self).__init__()
    self.linear1=nn.Linear(input,H)
    self.linear2=nn.Linear(H,output)
  
  def forward(self,x):
    x=torch.sigmoid(self.linear1(x))  
    x=torch.sigmoid(self.linear2(x))
    return x

In [23]:
import csv
import numpy as np
import os

class PoseClassifier(object):
  """Classifies pose landmarks."""

  def __init__(self,
               pose_samples_folder,
               pose_embedder,
               file_extension='csv',
               file_separator=',',
               n_landmarks=33,
               n_dimensions=3,
               axes_weights=(1., 1., 0.2),
               batch_size=64):
    self._pose_embedder = pose_embedder
    self._n_landmarks = n_landmarks
    self._n_dimensions = n_dimensions
    self._axes_weights = axes_weights

    self._pose_samples = self._load_pose_samples(pose_samples_folder,
                                                 file_extension,
                                                 file_separator,
                                                 n_landmarks,
                                                 n_dimensions,
                                                 pose_embedder)

    print('Successfully loaded {} poses.'.format(len(self._pose_samples)))
    random.shuffle(self._pose_samples)
    

    ## for reproducibility
    torch.manual_seed(0)
    np.random.seed(0)

    class_labels = ['burglary', 'normal']
    X = np.asarray([pose_sample.embedding for pose_sample in self._pose_samples], dtype=np.float32)
    X = X.reshape((X.shape[0], -1))
    Y = np.asarray([class_labels.index(pose_sample.class_name) for pose_sample in self._pose_samples], dtype=np.float32)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
    print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

    ## Instantiating classifier
    input_dim = 25 * 3   # how many Variables are in the dataset
    hidden_dim_0 = 128  # hidden layers
    hidden_dim_1 = 32
    output_dim = 1   # number of classes
    self.device = torch.device('cuda')
    self.net = Net_2(input_dim, hidden_dim_0,output_dim).to(self.device)

    class Data(Dataset):
        def __init__(self, X, Y):
            self.X=torch.from_numpy(X)
            self.Y=torch.from_numpy(Y)
            self.len=self.X.shape[0]
        def __getitem__(self,index):      
            return self.X[index], self.Y[index]
        def __len__(self):
            return self.len
    
    self.train_loader=DataLoader(dataset=Data(X_train,Y_train),batch_size=batch_size)
    self.test_loader=DataLoader(dataset=Data(X_test,Y_test),batch_size=batch_size)

  def _load_pose_samples(self,
                         pose_samples_folder,
                         file_extension,
                         file_separator,
                         n_landmarks,
                         n_dimensions,
                         pose_embedder):
    """Loads pose samples from a given folder.
    
    Required folder structure:
      normal.csv
      burglary.csv
      ...

    Required CSV structure:
      sample_00001,x1,y1,z1,x2,y2,z2,....
      sample_00002,x1,y1,z1,x2,y2,z2,....
      ...
    """
    # Each file in the folder represents one pose class.
    file_names = [name for name in os.listdir(pose_samples_folder) if name.endswith(file_extension)]

    pose_samples = []
    for file_name in file_names:
      # Use file name as pose class name.
      class_name = file_name[:-(len(file_extension) + 1)]
      
      # Parse CSV.
      with open(os.path.join(pose_samples_folder, file_name)) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=file_separator)
        for row in csv_reader:
          assert len(row) == n_landmarks * n_dimensions + 1, 'Wrong number of values: {}'.format(len(row))
          landmarks = np.array(row[1:], np.float32).reshape([n_landmarks, n_dimensions])
          pose_samples.append(PoseSample(
              name=row[0],
              landmarks=landmarks,
              class_name=class_name,
              embedding=pose_embedder(landmarks),
          ))

    return pose_samples
  
  def train_classifier(self, lr=0.01, num_epochs=100):
    
    ## Defining optimizer and loss function
    criterion = nn.BCELoss()
    optimizer = optim.SGD(self.net.parameters(), lr=0.01, momentum=0.9)

    ## Defining Training Parameters
    running_loss_list = [] # list to store running loss in the code below

    for epoch in range(num_epochs):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(self.train_loader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(self.device), labels.to(self.device)

            # forward + backward + optimize
            outputs = self.net(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            
            # zero the parameter gradients
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        if epoch % 100 == 99:
            print('[{}, {}] loss: {:.5f}'.format(epoch + 1, i + 1, running_loss / len(self.train_loader)))
            # self.test()
        running_loss_list.append(running_loss)
        running_loss = 0.0

    print('Finished Training')

    # plot loss curve
    # step = np.linspace(0,num_epochs,min(num_epochs, 1000))
    # plt.plot(step,np.array(running_loss_list))

  def test(self, model_path='./net.pth'):
    # net = Net()
    # net.load_state_dict(torch.load(model_path))
    # uncomment if batchnorm layers or dropout layers are used
    # net.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for data in self.test_loader:
            inputs, labels = data
            inputs, labels = inputs.to(self.device), labels.to(self.device)
            outputs = self.net(inputs)

            predicted = torch.round(outputs).reshape(-1)
            labels = labels.float()
            total += labels.size(0)
            
            correct += (predicted == labels).sum().item()

    ### complete the code to compute accuracy and store it as the variable acc 
    acc = correct / total ## stores the accuracy computed in the above loop 
    print('Accuracy of the network on the test poses: %f %%' % (acc * 100))


## Classification smoothing

In [22]:
class EMADictSmoothing(object):
  """Smoothes pose classification."""

  def __init__(self, window_size=10, alpha=0.2):
    self._window_size = window_size
    self._alpha = alpha

    self._data_in_window = []

  def __call__(self, data):
    """Smoothes given pose classification.

    Smoothing is done by computing Exponential Moving Average for every pose
    class observed in the given time window. Missed pose classes arre replaced
    with 0.
    
    Args:
      data: Dictionary with pose classification. Sample:
          {
            'normal': 8,
            'burglary': 2,
          }

    Result:
      Dictionary in the same format but with smoothed and float instead of
      integer values. Sample:
        {
          'normal': 8.3,
          'burglary': 1.7,
        }
    """
    # Add new data to the beginning of the window for simpler code.
    self._data_in_window.insert(0, data)
    self._data_in_window = self._data_in_window[:self._window_size]

    # Get all keys.
    keys = set([key for data in self._data_in_window for key, _ in data.items()])

    # Get smoothed values.
    smoothed_data = dict()
    for key in keys:
      factor = 1.0
      top_sum = 0.0
      bottom_sum = 0.0
      for data in self._data_in_window:
        value = data[key] if key in data else 0.0

        top_sum += factor * value
        bottom_sum += factor

        # Update factor.
        factor *= (1.0 - self._alpha)

      smoothed_data[key] = top_sum / bottom_sum

    return smoothed_data

## Repetition counter

In [8]:
class RepetitionCounter(object):
  """Counts number of repetitions of given target pose class."""

  def __init__(self, class_name, enter_threshold=6, exit_threshold=4):
    self._class_name = class_name

    # If pose counter passes given threshold, then we enter the pose.
    self._enter_threshold = enter_threshold
    self._exit_threshold = exit_threshold

    # Either we are in given pose or not.
    self._pose_entered = False

    # Number of times we exited the pose.
    self._n_repeats = 0

  @property
  def n_repeats(self):
    return self._n_repeats

  def __call__(self, pose_classification):
    """Counts number of repetitions happend until given frame.

    We use two thresholds. First you need to go above the higher one to enter
    the pose, and then you need to go below the lower one to exit it. Difference
    between the thresholds makes it stable to prediction jittering (which will
    cause wrong counts in case of having only one threshold).
    
    Args:
      pose_classification: Pose classification dictionary on current frame.
        Sample:
          {
            'normal': 8.3,
            'burglary': 1.7,
          }

    Returns:
      Integer counter of repetitions.
    """
    # Get pose confidence.
    pose_confidence = 0.0
    if self._class_name in pose_classification:
      pose_confidence = pose_classification[self._class_name]

    # On the very first frame or if we were out of the pose, just check if we
    # entered it on this frame and update the state.
    if not self._pose_entered:
      self._pose_entered = pose_confidence > self._enter_threshold
      return self._n_repeats

    # If we were in the pose and are exiting it, then increase the counter and
    # update the state.
    if pose_confidence < self._exit_threshold:
      self._n_repeats += 1
      self._pose_entered = False

    return self._n_repeats

# Step 1: Build classifier

## Bootstrap images

In [10]:
from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/My Drive/Colab Notebooks/Capstone

Mounted at /content/drive/
/content/drive/My Drive/Colab Notebooks/Capstone


In [11]:
# Required structure of the images_in_folder:
#
#   poses_images_in/
#     burglary/
#       image_001.jpg
#       image_002.jpg
#       ...
#     normal/
#       image_001.jpg
#       image_002.jpg
#       ...
#     ...
bootstrap_images_in_folder = 'poses_images_in'

# Output folders for bootstrapped images and CSVs.
bootstrap_images_out_folder = 'poses_images_out'
bootstrap_csvs_out_folder = 'poses_csvs_out'

## Train

In [24]:
# Transforms pose landmarks into embedding.
pose_embedder = FullBodyPoseEmbedder()



for lr in [0.1, 0.05, 0.01]:
    for batch_size in [100, 250, 500, 1000]:
        print(lr, batch_size)
        # Classifies give pose against database of poses.
        pose_classifier = PoseClassifier(
                pose_samples_folder=bootstrap_csvs_out_folder,
                pose_embedder=pose_embedder,
                batch_size=batch_size)
        pose_classifier.train_classifier(lr=lr,num_epochs=1000)
        pose_classifier.test()

0.1 100
Successfully loaded 2367 poses.
(1893, 75) (474, 75) (1893,) (474,)
[100, 19] loss: 0.29507
[200, 19] loss: 0.24490
[300, 19] loss: 0.18009
[400, 19] loss: 0.17431
[500, 19] loss: 0.15208
[600, 19] loss: 0.18103
[700, 19] loss: 0.17999
[800, 19] loss: 0.12991
[900, 19] loss: 0.11245
[1000, 19] loss: 0.16405
Finished Training
Accuracy of the network on the test poses: 80.590717 %
0.1 250
Successfully loaded 2367 poses.
(1893, 75) (474, 75) (1893,) (474,)
[100, 8] loss: 0.14279
[200, 8] loss: 0.05407
[300, 8] loss: 0.03195
[400, 8] loss: 0.02248
[500, 8] loss: 0.01824
[600, 8] loss: 0.01377
[700, 8] loss: 0.01157
[800, 8] loss: 0.00999
[900, 8] loss: 0.00876
[1000, 8] loss: 0.00781
Finished Training
Accuracy of the network on the test poses: 83.966245 %
0.1 500
Successfully loaded 2367 poses.
(1893, 75) (474, 75) (1893,) (474,)
[100, 4] loss: 0.20430
[200, 4] loss: 0.11204
[300, 4] loss: 0.06985
[400, 4] loss: 0.04991
[500, 4] loss: 0.03820
[600, 4] loss: 0.03045
[700, 4] loss: 0

# Step 2: Classification

**Important!!** Check that you are using the same classification parameters as while building classifier.

In [ ]:
# Upload your video.
uploaded = files.upload()
os.listdir('.')

Saving Burglary066_x264.mp4 to Burglary066_x264.mp4


['Pose classification (basic).ipynb',
 'poses_images_in',
 'poses_csvs_out',
 'poses_images_out',
 'Pose classification (extended).ipynb',
 'Burglary066_x264.mp4']

In [ ]:
# Specify your video name and target pose class to count the repetitions.
video_path = 'Burglary066_x264.mp4'
class_name='burglary'
out_video_path = 'sample-out.mov'

In [ ]:
# Open the video.
import cv2

video_cap = cv2.VideoCapture(video_path)

# Get some video parameters to generate output video with classificaiton.
video_n_frames = video_cap.get(cv2.CAP_PROP_FRAME_COUNT)
video_fps = video_cap.get(cv2.CAP_PROP_FPS)
video_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [ ]:
# Initilize tracker, classifier and counter.
# Do that before every video as all of them have state.
from mediapipe.python.solutions import pose as mp_pose


# Folder with pose class CSVs. That should be the same folder you using while
# building classifier to output CSVs.
pose_samples_folder = 'poses_csvs_out'

# Initialize tracker.
pose_tracker = mp_pose.Pose()

# Initialize embedder.
pose_embedder = FullBodyPoseEmbedder()

# Initialize classifier.
# Check that you are using the same parameters as during bootstrapping.
pose_classifier = PoseClassifier(
    pose_samples_folder=pose_samples_folder,
    pose_embedder=pose_embedder,
    top_n_by_max_distance=30,
    top_n_by_mean_distance=10)

# # Uncomment to validate target poses used by classifier and find outliers.
# outliers = pose_classifier.find_pose_sample_outliers()
# print('Number of pose sample outliers (consider removing them): ', len(outliers))

# Initialize EMA smoothing.
pose_classification_filter = EMADictSmoothing(
    window_size=10,
    alpha=0.2)

# Initialize counter.
repetition_counter = RepetitionCounter(
    class_name=class_name,
    enter_threshold=6,
    exit_threshold=4)

# Initialize renderer.

 = PoseClassificationVisualizer(
    class_name=class_name,
    plot_x_max=video_n_frames,
    # Graphic looks nicer if it's the same as `top_n_by_mean_distance`.
    plot_y_max=10)

In [ ]:
# Run classification on a video.
import os
import tqdm

from mediapipe.python.solutions import drawing_utils as mp_drawing


# Open output video.
out_video = cv2.VideoWriter(out_video_path, cv2.VideoWriter_fourcc(*'mp4v'), video_fps, (video_width, video_height))

frame_idx = 0
output_frame = None
with tqdm.tqdm(total=video_n_frames, position=0, leave=True) as pbar:
  while True:
    # Get next frame of the video.
    success, input_frame = video_cap.read()
    if not success:
      break

    # Run pose tracker.
    input_frame = cv2.cvtColor(input_frame, cv2.COLOR_BGR2RGB)
    result = pose_tracker.process(image=input_frame)
    pose_landmarks = result.pose_landmarks

    # Draw pose prediction.
    output_frame = input_frame.copy()
    if pose_landmarks is not None:
      mp_drawing.draw_landmarks(
          image=output_frame,
          landmark_list=pose_landmarks,
          connections=mp_pose.POSE_CONNECTIONS)
    
    if pose_landmarks is not None:
      # Get landmarks.
      frame_height, frame_width = output_frame.shape[0], output_frame.shape[1]
      pose_landmarks = np.array([[lmk.x * frame_width, lmk.y * frame_height, lmk.z * frame_width]
                                 for lmk in pose_landmarks.landmark], dtype=np.float32)
      assert pose_landmarks.shape == (33, 3), 'Unexpected landmarks shape: {}'.format(pose_landmarks.shape)

      # Classify the pose on the current frame.
      pose_classification = pose_classifier(pose_landmarks)

      # Smooth classification using EMA.
      pose_classification_filtered = pose_classification_filter(pose_classification)

      # Count repetitions.
      repetitions_count = repetition_counter(pose_classification_filtered)
    else:
      # No pose => no classification on current frame.
      pose_classification = None

      # Still add empty classification to the filter to maintaing correct
      # smoothing for future frames.
      pose_classification_filtered = pose_classification_filter(dict())
      pose_classification_filtered = None

      # Don't update the counter presuming that person is 'frozen'. Just
      # take the latest repetitions count.
      repetitions_count = repetition_counter.n_repeats

    # Draw classification plot and repetition counter.
    output_frame = pose_classification_visualizer(
        frame=output_frame,
        pose_classification=pose_classification,
        pose_classification_filtered=pose_classification_filtered,
        repetitions_count=repetitions_count)

    # Save the output frame.
    out_video.write(cv2.cvtColor(np.array(output_frame), cv2.COLOR_RGB2BGR))

    # Show intermediate frames of the video to track progress.
    if frame_idx % 50 == 0:
      show_image(output_frame)

    frame_idx += 1
    pbar.update()

# Close output video.
out_video.release()

# Release MediaPipe resources.
pose_tracker.close()

# Show the last frame of the video.
if output_frame is not None:
  show_image(output_frame)

In [ ]:
# Download generated video
files.download(out_video_path)